In [ ]:
cd /content/drive/My Drive/MURA/mura_unzipped/MURA-v1.1

/content/drive/My Drive/MURA/mura_unzipped/MURA-v1.1


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from skimage.io import imshow
from pathlib import Path
import pandas as pd

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/content/drive/My Drive/MURA/mura_unzipped"))


['MURA-v1.1']


In [ ]:
dataset_root = Path('../MURA-v1.1')

In [ ]:
list(dataset_root.iterdir())

[PosixPath('../MURA-v1.1/valid_labeled_studies.csv'),
 PosixPath('../MURA-v1.1/valid_image_paths.csv'),
 PosixPath('../MURA-v1.1/valid'),
 PosixPath('../MURA-v1.1/train_labeled_studies.csv'),
 PosixPath('../MURA-v1.1/train'),
 PosixPath('../MURA-v1.1/train_image_paths.csv'),
 PosixPath('../MURA-v1.1/v_train'),
 PosixPath('../MURA-v1.1/v_valid'),
 PosixPath('../MURA-v1.1/v_test')]

In [ ]:
df = pd.read_csv('train_image_paths.csv', header=None, names=['filename'])
df.head()

,filename
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [ ]:
df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
df.head()

,filename,class
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive


In [ ]:
def generate_df(dataset_root, csv_name):
    df = pd.read_csv(dataset_root/csv_name, header=None, names=['filename'])
    df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
    return df

In [ ]:
list(dataset_root.parent.iterdir())

[PosixPath('../MURA-v1.1')]

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255)
train_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'train_image_paths.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')
valid_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'valid_image_paths.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 36803 validated image filenames belonging to 2 classes.
Found 3197 validated image filenames belonging to 2 classes.


In [ ]:
densenet = tf.keras.applications.DenseNet169(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

51879936/51877672 [==============================] - 1s 0us/step


In [ ]:
densenet.trainable = False

In [ ]:
densenet.summary()

Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [ ]:
model = tf.keras.models.Sequential([
    densenet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 7, 7, 1664)        12642880  
_________________________________________________________________
flatten (Flatten)            (None, 81536)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 81537     
Total params: 12,724,417
Trainable params: 81,537
Non-trainable params: 12,642,880
_________________________________________________________________


In [ ]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(train_gen, epochs=5, validation_data=valid_gen, use_multiprocessing=True)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
1151/1151 [==============================] - 24120s 21s/step - loss: 1.2598 - accuracy: 0.6753 - val_loss: 1.4479 - val_accuracy: 0.6678
Epoch 2/5
1151/1151 [==============================] - 338s 294ms/step - loss: 1.2799 - accuracy: 0.7174 - val_loss: 1.3209 - val_accuracy: 0.7038
Epoch 3/5
1151/1151 [==============================] - 315s 274ms/step - loss: 1.1257 - accuracy: 0.7449 - val_loss: 1.5076 - val_accuracy: 0.6831
Epoch 4/5
1151/1151 [==============================] - 314s 273ms/step - loss: 1.0259 - accuracy: 0.7646 - val_loss: 1.6386 - val_accuracy: 0.6788
Epoch 5/5
1151/1151 [==============================] - 307s 267ms/step - loss: 1.0430 - accuracy: 0.7687 - val_loss: 1.9668 - val_accuracy: 0.6828


In [ ]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# from keras.models import load_model

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model_new = load_model('model.h5')
model_new.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 7, 7, 1664)        12642880  
_________________________________________________________________
flatten (Flatten)            (None, 81536)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 81537     
Total params: 12,724,417
Trainable params: 81,537
Non-trainable params: 12,642,880
_________________________________________________________________
